<img src="./images/03-faq.png" width=700px>


In this example we will see an example of RAG - Retrieval Augmented Generation.

We have 'retrieved' some data and stored in the FAQ variable. In this demo, this is hard coded but there are a multitude of ways of RETRIEVING data to AUGMENT the prompt and consequent GENERATION.

OpenAi does not have any domain specifice knowledge. In lieu of a fine tuned model on our domain, we can AUGMENT the prompt with retrieved data.

RAG is not just asking questions of documents using vector search - it is augmenting the LLM request with additonal data in lieu of fine tuning the model.

We are going to give it a list of FAQs so that we can use a ChatBot to ask questions.


In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

c:\Users\mrcra\Desktop\django-con-eu\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_AP#I_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY not set
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


In [4]:
# A request to the LLM is stateless so we will always need to pass all the data that is needed each time.

# `history` is just that - a record of what has gone on before so that the LLM can have context to answer the query.


# We will use GRADIO as our UI.
def chat(message, history):
    # history is part of the gradio ChatInterface and it stores previous answers
    messages = (
        [{"role": "system", "content": system_message}]
        # + history ## groq adds metadata and causes error
        + [{"role": "user", "content": message}]
    )
    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)
    # ====================
    # AI bit
    stream = client.chat.completions.create(model=MODEL, messages=messages, stream=True)

    # Just UI implementation
    response = ""
    for stream_so_far in stream:
        response += stream_so_far.choices[0].delta.content or ""
        yield response

In [5]:
# We set up the backround scenario in the system messaage.
# In some frameworks this is called 'background' or 'character'.

system_message = """You are a helpful assistant for the Django Conference. """

In [6]:
# FAQ is just data extracted from a source and inserted into the system message.
# RAG - Retrieval Augmented Generation.
# This does not need to be a vector search but additional information we pass to the LLM.

# ORIGINAL FAQ about a shop
FAQ_SHOP = [
    "We only sell shoes.",
    "Our opening hours are Monday to Friday from 9am to 5pm.",
    "We are located at 123 Main Street, Brighton",
    "We specialise in red shoes but have all colours",
    "Our VAT rate is 20 percent and is applicable on all sales",
    "We only accept card payments",
]

FAQ for DjangoCon 2025

In [7]:
# FAQ is just data extracted from a source and inserted into the system message.
# RAG - Retrieval Augmented Generation.
# This does not need to be a vector search but additional RETREIVED information we pass to the LLM to AUGMENT the prompt prior to GENERATING the response.
FAQ = [
    "DjangoCon 2025 is taking place in Dublin",
    "The dates are 23rd-27th April 2025",
    "The venue is DjangoCon Europe 2025 will be held in Dublin! The capital of the Republic of Ireland, one of Europe’s biggest tech hubs, is a lively city with extraordinarily rich and diverse food, culture, history and art lovers - the number of museums and restaurants can certainly tickle most tastes.",
    "The conference will take place at Talbot Hotel Stillorgan, Dublin",
    "As an EU/EEA resident, you do not require a visa to travel to Ireland. You can enter the country freely under the Common Travel Area (CTA) agreements.",
    """
    Documents to Carry
    Even though a visa is not required, ensure you carry:
    - A valid passport or national ID card.
    - Proof of accommodation (e.g., hotel bookings or host details).
    - Your DjangoCon Europe ticket or event confirmation.
    - Travel insurance (optional but recommended).
    """,
    "DjangoCon Europe 2025 offers grants so that those who might otherwise not be able to attend won't hesitate to participate. Some expenses for the conference attendance (which could include travel, hotel, registration, etc) will be covered for opportunity grant recipients.",
    "The sponsors are Ambient, Caktus, Monit to name but a few.",
    """
    Mentorship Program
    Are you an experienced speaker eager to share your knowledge and guide aspiring speakers? Or are you planning to submit a proposal to DjangoCon Europe 2025 and looking for expert advice to craft a standout submission?
    If you answered 'yes' to either question, this mentorship program is perfect for you!
    """,
    """ 
    For Mentees This is How the Program Works
    Our aim is to pair each mentee with a dedicated mentor to foster one-on-one guidance. To simplify scheduling, we prioritize matching participants in the same timezone or country whenever possible.
    """,
    """
    A live recording of a 'The Real Python Podcast' episode with audience participation.

    The Real Python Podcast is a weekly podcast hosted by Christopher Bailey with interviews, coding tips, and conversations with guests from the Python community. Bi-weekly, the episodes are cohosted by Christopher Trudeau where together the Christophers cover recent content from the PyCoders newsletter and happenings in the Python world.

    The DjangoCon session would be similar to PyCoders episodes where we cover interesting recent Python articles. For DjangoCon, content would be Django focused, with the list of articles determined closer to the conference date. Questions from the audience could be on the topics covered and/or in a AMA format.""",
    """Sprints
So you have listened to all the exciting topics on django for three days and now feel motivated do something about it. What could be a better motivation than contributing to the very open source software you just listened to?

On Saturday and Sunday, you have the opportunity to do exactly that. This is your opportunity to contribute to Open Source software. Even if you haven't contributed before, this is also an option to take your first steps. In the sprint, you can contribute to Django as you see fit, and if you need help, there will a pool of experienced contributors to guide you.

Main Venue - Hosting our Sprints
Talbot Hotel Stillorgan

Time
Saturday - 9AM to 6PM
Sunday - 9AM to 6PM
The Sprints are free for all conference participants. However, as the Sprints venue does not have unlimited capacity, and we would like to prepare a precise amount of food and drinks, please keep your participation status updated on the ticketing website (after buying your ticket).

We will be hosting our sprints at Talbot Hotel Stillorgan. Get ready for an amazing conference experience in beautiful Dublin!

Check the map to the venue""",
]

In [8]:
# Create the base system message - 'character and instructions' along with retrieved data.
system_message += "\n" + "\n".join(FAQ_SHOP) + "\n" + "\n".join(FAQ)

In [9]:
# We use Gradio for a chat interface
# prompt: I am interested in green belts, probably coming Sunday and will have cash. Is this OK?

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant for the Django Conference. \nWe only sell shoes.\nOur opening hours are Monday to Friday from 9am to 5pm.\nWe are located at 123 Main Street, Brighton\nWe specialise in red shoes but have all colours\nOur VAT rate is 20 percent and is applicable on all sales\nWe only accept card payments\nDjangoCon 2025 is taking place in Dublin\nThe dates are 23rd-27th April 2025\nThe venue is DjangoCon Europe 2025 will be held in Dublin! The capital of the Republic of Ireland, one of Europe’s biggest tech hubs, is a lively city with extraordinarily rich and diverse food, culture, history and art lovers - the number of museums and restaurants can certainly tickle most tastes.\nThe conference will take place at Talbot Hotel Stillorgan, Dublin\nAs an EU/EEA resident, you do not require a visa to travel to Ireland. You can enter the country freely under the Common Travel Area (CTA) agreements.\n\n    Documents to 